In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import numpy
MNIST = input_data.read_data_sets("./MNIST_data", one_hot = True)
    

Extracting ./MNIST_data\train-images-idx3-ubyte.gz
Extracting ./MNIST_data\train-labels-idx1-ubyte.gz
Extracting ./MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:
def get_train_batch_func(batch_size, fake_data = False):
    #"""Return the next `batch_size` examples from this data set."""
    self = MNIST.train
    if fake_data:
        fake_image = [1] * 784
        if self.one_hot:
            fake_label = [1] + [0] * 9
        else:
            fake_label = 0
        return [fake_image for _ in xrange(batch_size)], [
            fake_label for _ in xrange(batch_size)
        ]
    start = self._index_in_epoch
    self._index_in_epoch += 128       #원래는 batch_size였음
    if self._index_in_epoch > self._num_examples:
        
        if self._index_in_epoch - self._num_examples < 128: #마지막으로 N % 128 개인 경우
            self._index_in_epoch = self._num_examples 
            
        else:
            # Finished epoch
            self._epochs_completed += 1
            # Shuffle the data
            perm = numpy.arange(self._num_examples)
            numpy.random.shuffle(perm)
            self._images = self._images[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = 128  #원래는 batch_size 였음
            assert batch_size <= self._num_examples
    
    end = self._index_in_epoch
    return self._images[start:end], self._labels[start:end]
    


def exec1(l_rate, t_epochs, b_size, directory_name, act_func, optimizer_func, get_train_batch):
    #############################exercise2 : double layered fully connected neural network with softmax classifier###########################
    #parameters ########exercise2-1: should change parameters and compare results
    learning_rate =l_rate #learning_rate = 0.001
    training_epochs = t_epochs #training_epochs = 2
    batch_size = b_size #batch_size = 100

    SUMMARY_DIR = directory_name
    
    #load mnist dataset
    
    with tf.name_scope('input') as scope:
        X = tf.placeholder(tf.float32, [None, 784], name = 'image')
        y = tf.placeholder(tf.float32, [None, 10], name = 'label')    
    
    with tf.variable_scope('layer1') as scope:
        W1 = tf.get_variable("W", shape = [784, 512], initializer = tf.contrib.layers.xavier_initializer())
        b1 = tf.Variable(tf.random_normal([512]))
        L1 = act_func(tf.add(tf.matmul(X, W1), b1))#########################################ex1-2act_func
    
        tf.summary.histogram("X", X)
        tf.summary.histogram("weights", W1)
        tf.summary.histogram("bias", b1)
        tf.summary.histogram("layer", L1)    
    
    with tf.variable_scope('layer12') as scope:
        W12 = tf.get_variable("W", shape = [512, 512], initializer = tf.contrib.layers.xavier_initializer())
        b12 = tf.Variable(tf.random_normal([512]))
        L12 = act_func(tf.add(tf.matmul(L1, W12), b12))#########################################ex1-2act_func
    
        tf.summary.histogram("weights", W12)
        tf.summary.histogram("bias", b12)
        tf.summary.histogram("layer", L12)

    with tf.variable_scope('layer2') as scope:
        W2 = tf.get_variable("W", shape = [512, 10], initializer = tf.contrib.layers.xavier_initializer())
        b2 = tf.Variable(tf.random_normal([10]))
        y_ = tf.add(tf.matmul(L12, W2), b2)
    
        tf.summary.histogram("weights", W2)
        tf.summary.histogram("bias", b2)
        tf.summary.histogram("logits", y_)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_, labels = y))
    optimizer = optimizer_func(learning_rate = learning_rate).minimize(loss) #################################ex1-2optimizer

    tf.summary.scalar("loss", loss)

    #merge all summaries
    summary = tf.summary.merge_all()

    global_step = 0
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
        for epoch in range(training_epochs):
            total_batch = int(MNIST.train.num_examples / batch_size)
            avg_loss = 0
        
            for i in range(total_batch):
                batch_xs, batch_ys = get_train_batch(batch_size)##########################exercise2-3 get_train_batch구현,적용
                feed_dict = {X: batch_xs, y: batch_ys}
                s, I, _ = sess.run([summary, loss, optimizer], feed_dict = feed_dict)
                writer.add_summary(s, global_step = global_step)#무언가 문제가 있음 원래는 파라미터가 s, I, glbal_step이었는데 I 뺌 일단
                global_step += 1
                avg_loss += I
            print('Epoch:', '%02d' % (epoch + 1), 'loss = ', '{:6f}'.format(avg_loss / total_batch))
    
        correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        acc = sess.run(accuracy, feed_dict = {X: MNIST.test.images, y: MNIST.test.labels})
        print('Test accuracy:', acc)
    
    tf.reset_default_graph() ## layer 이름의 중복 문제 해결을 위해 추가.    
#l_rate, t_epochs, b_size,
#learning_rate = 0.001
#training_epochs = 2
#batch_size = 100
dir_name0 = './mnist3'
dir_name1 = './dir_of_log1'
#############################exercise2
exec1(0.001, 15, 100, dir_name0, tf.nn.softmax, tf.train.GradientDescentOptimizer, MNIST.train.next_batch) #exercise2-1
exec1(0.001, 15, 150, dir_name0, tf.nn.softmax, tf.train.GradientDescentOptimizer, MNIST.train.next_batch) #exercise2-1-1
exec1(0.005, 15, 100, dir_name0, tf.nn.softmax, tf.train.GradientDescentOptimizer, MNIST.train.next_batch) #exercise2-1-1
exec1(0.005, 20, 100, dir_name0, tf.nn.softmax, tf.train.GradientDescentOptimizer, MNIST.train.next_batch) #exercise2-1-1

#############################exercise2-1
exec1(0.001, 15, 100, './dir_of_log01', tf.nn.relu, tf.train.AdamOptimizer, MNIST.train.next_batch) #exercise3-1-2
exec1(0.001, 15, 100, './dir_of_log02', tf.nn.relu, tf.train.GradientDescentOptimizer, MNIST.train.next_batch) #exercise3-1-2
exec1(0.001, 15, 100, './dir_of_log03', tf.nn.sigmoid, tf.train.AdamOptimizer, MNIST.train.next_batch) #exercise3-1-2
exec1(0.001, 15, 100, './dir_of_log04', tf.nn.sigmoid, tf.train.GradientDescentOptimizer, MNIST.train.next_batch) #exercise3-1-2

#############################exercise3
exec1(0.001, 15, 100, './dir_of_log11', tf.nn.relu, tf.train.AdamOptimizer, MNIST.train.next_batch) #exercise3-1-2
exec1(0.001, 15, 100, './dir_of_log12', tf.nn.relu, tf.train.AdamOptimizer, get_train_batch_func) #exercise3-2-3

Epoch: 01 loss =  2.425958
Epoch: 02 loss =  2.416129
Epoch: 03 loss =  2.407149
Epoch: 04 loss =  2.398926
Epoch: 05 loss =  2.391381
Epoch: 06 loss =  2.384446
Epoch: 07 loss =  2.378061
Epoch: 08 loss =  2.372173
Epoch: 09 loss =  2.366739
Epoch: 10 loss =  2.361721
Epoch: 11 loss =  2.357083
Epoch: 12 loss =  2.352791
Epoch: 13 loss =  2.348817
Epoch: 14 loss =  2.345137
Epoch: 15 loss =  2.341728
Test accuracy: 0.0958
Epoch: 01 loss =  2.857259
Epoch: 02 loss =  2.829148
Epoch: 03 loss =  2.802808
Epoch: 04 loss =  2.779270
Epoch: 05 loss =  2.753926
Epoch: 06 loss =  2.732126
Epoch: 07 loss =  2.711036
Epoch: 08 loss =  2.691214
Epoch: 09 loss =  2.672051
Epoch: 10 loss =  2.654346
Epoch: 11 loss =  2.636300
Epoch: 12 loss =  2.619442
Epoch: 13 loss =  2.605752
Epoch: 14 loss =  2.589259
Epoch: 15 loss =  2.575592
Test accuracy: 0.0892
Epoch: 01 loss =  2.386607
Epoch: 02 loss =  2.352633
Epoch: 03 loss =  2.331308
Epoch: 04 loss =  2.319316
Epoch: 05 loss =  2.311857
Epoch: 06 l

In [3]:
#Supplement of exercise2
exec1(0.001, 15, 100, './dir_of_log20', tf.nn.relu, tf.train.AdamOptimizer, MNIST.train.next_batch) #exercise2-1
exec1(0.001, 15, 150, './dir_of_log20', tf.nn.relu, tf.train.AdamOptimizer, MNIST.train.next_batch) #exercise2-1-1
exec1(0.005, 15, 100, './dir_of_log20', tf.nn.relu, tf.train.AdamOptimizer, MNIST.train.next_batch) #exercise2-1-1
exec1(0.005, 20, 100, './dir_of_log20', tf.nn.relu, tf.train.AdamOptimizer, MNIST.train.next_batch) #exercise2-1-1    

Epoch: 01 loss =  0.275439
Epoch: 02 loss =  0.093915
Epoch: 03 loss =  0.061334
Epoch: 04 loss =  0.042007
Epoch: 05 loss =  0.028348
Epoch: 06 loss =  0.024801
Epoch: 07 loss =  0.019204
Epoch: 08 loss =  0.018717
Epoch: 09 loss =  0.015804
Epoch: 10 loss =  0.013966
Epoch: 11 loss =  0.011437
Epoch: 12 loss =  0.012661
Epoch: 13 loss =  0.010399
Epoch: 14 loss =  0.008573
Epoch: 15 loss =  0.008139
Test accuracy: 0.9816
Epoch: 01 loss =  0.303015
Epoch: 02 loss =  0.107602
Epoch: 03 loss =  0.067260
Epoch: 04 loss =  0.048599
Epoch: 05 loss =  0.034331
Epoch: 06 loss =  0.024124
Epoch: 07 loss =  0.018206
Epoch: 08 loss =  0.017932
Epoch: 09 loss =  0.013319
Epoch: 10 loss =  0.012771
Epoch: 11 loss =  0.014019
Epoch: 12 loss =  0.010192
Epoch: 13 loss =  0.013513
Epoch: 14 loss =  0.006999
Epoch: 15 loss =  0.004064
Test accuracy: 0.9811
Epoch: 01 loss =  0.270252
Epoch: 02 loss =  0.109077
Epoch: 03 loss =  0.083485
Epoch: 04 loss =  0.069764
Epoch: 05 loss =  0.068152
Epoch: 06 l

In [4]:
#Supplement of exercise3
exec1(0.001, 15, 100, './dir_of_log11', tf.nn.relu, tf.train.GradientDescentOptimizer, MNIST.train.next_batch) #exercise3-1-2
exec1(0.001, 15, 100, './dir_of_log12', tf.nn.relu, tf.train.GradientDescentOptimizer, get_train_batch_func) #exercise3-2-3

Epoch: 01 loss =  2.044894
Epoch: 02 loss =  1.558917
Epoch: 03 loss =  1.203438
Epoch: 04 loss =  0.968119
Epoch: 05 loss =  0.813403
Epoch: 06 loss =  0.715878
Epoch: 07 loss =  0.641918
Epoch: 08 loss =  0.592449
Epoch: 09 loss =  0.550843
Epoch: 10 loss =  0.519210
Epoch: 11 loss =  0.494973
Epoch: 12 loss =  0.475159
Epoch: 13 loss =  0.457494
Epoch: 14 loss =  0.443039
Epoch: 15 loss =  0.428721
Test accuracy: 0.888
Epoch: 01 loss =  2.080448
Epoch: 02 loss =  1.603906
Epoch: 03 loss =  1.254522
Epoch: 04 loss =  1.010791
Epoch: 05 loss =  0.846787
Epoch: 06 loss =  0.741750
Epoch: 07 loss =  0.663419
Epoch: 08 loss =  0.607991
Epoch: 09 loss =  0.570051
Epoch: 10 loss =  0.532505
Epoch: 11 loss =  0.507253
Epoch: 12 loss =  0.485610
Epoch: 13 loss =  0.466803
Epoch: 14 loss =  0.451988
Epoch: 15 loss =  0.437407
Test accuracy: 0.8906
